In [12]:
import pandas as pd

In [13]:
train = pd.read_csv('train_tweets.csv')
print("training set:"%train.columns,train.shape,len(train))

training set: (31962, 3) 31962


In [14]:
test = pd.read_csv('test_tweets.csv')
print("test set:"%test.columns,test.shape,len(test))

test set: (17197, 2) 17197


In [15]:
import re
def clean_text(df,text_field):
    df[text_field]=df[text_field].str.lower()
    df[text_field]=df[text_field].apply(lambda elem:re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?","",elem))
    return df

In [16]:
test_clean=clean_text(test,"tweet")
train_clean=clean_text(train,"tweet")

In [17]:
from sklearn.utils import resample
train_minority = train_clean[train_clean.label==0]
train_majority = train_clean[train_clean.label==1]

train_minority_upsampled = resample(train_minority,replace=True,n_samples=len(train_majority),random_state=123)
train_upsampled = pd.concat([train_minority_upsampled,train_majority])
train_upsampled['label'].value_counts()

label
0    2242
1    2242
Name: count, dtype: int64

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd=Pipeline([('vect',CountVectorizer()),
                      ('tfidf',TfidfTransformer()),
                      ('nb',SGDClassifier()),])

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test = train_test_split(train_upsampled['tweet'],train_upsampled['label'],random_state=0)

In [20]:
model = pipeline_sgd.fit(X_train,y_train)

In [21]:
y_predict = model.predict(X_test)

In [22]:
from sklearn.metrics import f1_score
f1_score(y_test,y_predict)

0.8558558558558559